In [26]:
import pandas as pd
import numpy as np
import time
import random

In [27]:
# # data = pd.read_csv("D:/Udler downloads/train.csv")
data = pd.read_csv("../../../datasets/Churn_Modelling.csv")

In [28]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [29]:
data.nunique()

RowNumber          10000
CustomerId         10000
Surname             2932
CreditScore          460
Geography              3
Gender                 2
Age                   70
Tenure                11
Balance             6382
NumOfProducts          4
HasCrCard              2
IsActiveMember         2
EstimatedSalary     9999
Exited                 2
dtype: int64

In [30]:
data.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [31]:
DATA_PATH = "../../../datasets/Churn_Modelling.csv"

In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import nn, optim
from torch.autograd import Variable

import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [33]:
def load_and_standardize_data(path):
    # read in from csv
    df = pd.read_csv(path, sep=',')
    # replace nan with -99
    # df = df.fillna(-99)
    y = df['Exited']
    df.drop(['RowNumber', 'CustomerId', 'Surname','Geography',
       'Gender','Exited'],axis=1,inplace=True)
    xtrain,xtest,ytrain,ytest = train_test_split(df,y,stratify = y,test_size = 0.3,random_state = 4)
    # df = df.values.reshape(-1, df.shape[1]).astype('float32')
    # randomly split

    # standardize values
#     scaler = preprocessing.StandardScaler()
    scaler=preprocessing.MinMaxScaler()
    xtrain = scaler.fit_transform(xtrain)
    xtest= scaler.transform(xtest)
    return xtrain,xtest,scaler

In [34]:
from torch.utils.data import Dataset, DataLoader
class DataBuilder(Dataset):
    def __init__(self, path, train=True):
        self.X_train, self.X_test, self.standardizer = load_and_standardize_data(DATA_PATH)
        if train:
            # self.x = torch.from_numpy(self.X_train)
            self.x = torch.FloatTensor(self.X_train)
            self.len=self.x.shape[0]
        else:
            # self.x = torch.from_numpy(self.X_test)
            self.x = torch.FloatTensor(self.X_test)
            self.len=self.x.shape[0]
        del self.X_train
        del self.X_test
    def __getitem__(self,index):
        return self.x[index]
    def __len__(self):
        return self.len

In [35]:
traindata_set=DataBuilder(DATA_PATH, train=True)
testdata_set=DataBuilder(DATA_PATH, train=False)

trainloader=DataLoader(dataset=traindata_set,batch_size=20000)
testloader=DataLoader(dataset=testdata_set,batch_size=20000)

In [36]:
type(trainloader.dataset.x), type(testloader.dataset.x)

(torch.Tensor, torch.Tensor)

In [37]:
trainloader.dataset.x.shape, testloader.dataset.x.shape

(torch.Size([7000, 8]), torch.Size([3000, 8]))

In [38]:
trainloader.dataset.x

tensor([[0.8700, 0.2162, 0.9000,  ..., 1.0000, 1.0000, 0.3430],
        [0.8600, 0.2568, 1.0000,  ..., 1.0000, 0.0000, 0.6317],
        [0.6340, 0.2432, 0.3000,  ..., 1.0000, 1.0000, 0.9344],
        ...,
        [0.3860, 0.2568, 0.3000,  ..., 1.0000, 1.0000, 0.3946],
        [0.4180, 0.3243, 0.7000,  ..., 1.0000, 1.0000, 0.9504],
        [0.7880, 0.3378, 1.0000,  ..., 0.0000, 1.0000, 0.1211]])

In [39]:
testloader.dataset.x

tensor([[0.7860, 0.2973, 0.8000,  ..., 1.0000, 0.0000, 0.4745],
        [0.2760, 0.0541, 0.6000,  ..., 1.0000, 1.0000, 0.3320],
        [0.5320, 0.3378, 0.3000,  ..., 1.0000, 0.0000, 0.0938],
        ...,
        [0.5060, 0.1757, 0.8000,  ..., 1.0000, 1.0000, 0.8498],
        [0.5540, 0.3514, 0.6000,  ..., 0.0000, 0.0000, 0.1765],
        [0.5280, 0.2432, 0.1000,  ..., 1.0000, 0.0000, 0.7308]])

In [40]:
latent_dims = 2
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'

## VAE

In [41]:
class VAE2(nn.Module):
    def __init__(self,D_in,latent_dim,fn,H=10,H2=15):

        #Encoder
        super(VAE2,self).__init__()
        self.linear1=nn.Linear(D_in,H)
        self.lin_bn1 = nn.BatchNorm1d(num_features=H)
        self.linear2=nn.Linear(H,H2)
        self.lin_bn2 = nn.BatchNorm1d(num_features=H2)

#         # Latent vectors mu and sigma
        self.fc1 = nn.Linear(H2, latent_dim)
        self.bn1 = nn.BatchNorm1d(num_features=latent_dim)
        self.fc21 = nn.Linear(latent_dim, latent_dim)
        self.fc22 = nn.Linear(latent_dim, latent_dim)

#         # Sampling vector
        self.fc3 = nn.Linear(latent_dim, latent_dim)
        self.fc_bn3 = nn.BatchNorm1d(latent_dim)
        self.fc4 = nn.Linear(latent_dim, H2)
        self.fc_bn4 = nn.BatchNorm1d(H2)

#         # Decoder
        self.linear4=nn.Linear(H2,H)
        self.lin_bn4 = nn.BatchNorm1d(num_features=H)
        self.linear5=nn.Linear(H,D_in)
        self.lin_bn5 = nn.BatchNorm1d(num_features=D_in)
        self.prev_seed =random.uniform(0,1)

        self.relu = nn.ReLU()
        self.Tanh = nn.Tanh()
        
        self.fn=fn

    def encode(self, x):
        
        if self.fn=='tanh':
            lin1 = self.Tanh(self.lin_bn1(self.linear1(x)))
            lin2 =self.Tanh(self.lin_bn2(self.linear2(lin1)))
            
        if self.fn=='relu':
            lin1 = self.relu(self.lin_bn1(self.linear1(x)))
            lin2 =self.relu(self.lin_bn2(self.linear2(lin1)))
            

        fc1 = F.relu(self.bn1(self.fc1(lin2)))

        r1 = self.fc21(fc1)
        r2 = self.fc22(fc1)

        return r1, r2

    def decode(self, z):
        
        if self.fn=='tanh':
            fc3 = self.Tanh(self.fc_bn3(self.fc3(z)))
            fc4 =self.Tanh(self.fc_bn4(self.fc4(fc3)))

            lin4 =self.Tanh(self.lin_bn4(self.linear4(fc4)))
            #lin5 = self.Tanh(self.lin_bn5(self.linear5(lin4)))
        if self.fn=='relu':
            fc3 = self.relu(self.fc_bn3(self.fc3(z)))
            fc4 =self.relu(self.fc_bn4(self.fc4(fc3)))

            lin4 =self.relu(self.lin_bn4(self.linear4(fc4)))
            #lin5 = self.Tanh(self.lin_bn5(self.linear5(lin4)))
        
        return self.lin_bn5(self.linear5(lin4))
    

    def forward(self, x):
        mu,sigma = self.encode(x)
        prev_seed = self.prev_seed
#         print('vae prev_seed',prev_seed)
        z,self.prev_seed = reparameterize(mu,sigma,prev_seed)
#         print("z",z.shape)
#         znew = sampling_vector(z,H)
        return self.decode(z),mu,sigma

In [42]:
class VAE3(nn.Module):
    def __init__(self,D_in,latent_dim,fn,H=10,H2=15):

        #Encoder
        super(VAE3,self).__init__()
        self.linear1=nn.Linear(D_in,H)
        self.lin_bn1 = nn.BatchNorm1d(num_features=H)
        self.linear2=nn.Linear(H,H2)
        self.lin_bn2 = nn.BatchNorm1d(num_features=H2)
        self.linear3=nn.Linear(H2,H2)
        self.lin_bn3 = nn.BatchNorm1d(num_features=H2)

#         # Latent vectors mu and sigma
        self.fc1 = nn.Linear(H2, latent_dim)
        self.bn1 = nn.BatchNorm1d(num_features=latent_dim)
        self.fc21 = nn.Linear(latent_dim, latent_dim)
        self.fc22 = nn.Linear(latent_dim, latent_dim)

#         # Sampling vector
        self.fc3 = nn.Linear(latent_dim, latent_dim)
        self.fc_bn3 = nn.BatchNorm1d(latent_dim)
        self.fc4 = nn.Linear(latent_dim, H2)
        self.fc_bn4 = nn.BatchNorm1d(H2)

#         # Decoder
        self.linear4=nn.Linear(H2,H2)
        self.lin_bn4 = nn.BatchNorm1d(num_features=H2)
        self.linear5=nn.Linear(H2,H)
        self.lin_bn5 = nn.BatchNorm1d(num_features=H)
        self.linear6=nn.Linear(H,D_in)
        self.lin_bn6 = nn.BatchNorm1d(num_features=D_in)
        self.prev_seed =random.uniform(0,1)

        self.relu = nn.ReLU()
        self.Tanh = nn.Tanh()
        
        self.fn=fn

    def encode(self, x):
        
        if self.fn=='tanh':
            lin1 = self.Tanh(self.lin_bn1(self.linear1(x)))
            lin2 =self.Tanh(self.lin_bn2(self.linear2(lin1)))
            lin3 = self.Tanh(self.lin_bn3(self.linear3(lin2)))
        if self.fn=='relu':
            lin1 = self.relu(self.lin_bn1(self.linear1(x)))
            lin2 =self.relu(self.lin_bn2(self.linear2(lin1)))
            lin3 = self.relu(self.lin_bn3(self.linear3(lin2)))

        fc1 = F.relu(self.bn1(self.fc1(lin3)))

        r1 = self.fc21(fc1)
        r2 = self.fc22(fc1)

        return r1, r2

    def decode(self, z):
        
        if self.fn=='tanh':
            fc3 = self.Tanh(self.fc_bn3(self.fc3(z)))
            fc4 =self.Tanh(self.fc_bn4(self.fc4(fc3)))

            lin4 =self.Tanh(self.lin_bn4(self.linear4(fc4)))
            lin5 = self.Tanh(self.lin_bn5(self.linear5(lin4)))
        if self.fn=='relu':
            fc3 = self.relu(self.fc_bn3(self.fc3(z)))
            fc4 =self.relu(self.fc_bn4(self.fc4(fc3)))

            lin4 =self.relu(self.lin_bn4(self.linear4(fc4)))
            lin5 = self.relu(self.lin_bn5(self.linear5(lin4)))
        
        return self.lin_bn6(self.linear6(lin5))
    
    

    def forward(self, x):
        mu,sigma = self.encode(x)
        prev_seed = self.prev_seed
#         print('vae prev_seed',prev_seed)
        z,self.prev_seed = reparameterize(mu,sigma,prev_seed)
#         print("z",z.shape)
#         znew = sampling_vector(z,H)
        return self.decode(z),mu,sigma

In [43]:
class VAE4(nn.Module):
    def __init__(self,D_in,latent_dim,fn,H=10,H2=15,H3=20):

        #Encoder
        super(VAE4,self).__init__()
        self.linear1=nn.Linear(D_in,H)
        self.lin_bn1 = nn.BatchNorm1d(num_features=H)
        self.linear2=nn.Linear(H,H2)
        self.lin_bn2 = nn.BatchNorm1d(num_features=H2)
        self.linear3=nn.Linear(H2,H3)
        self.lin_bn3 = nn.BatchNorm1d(num_features=H3)
        self.linear4=nn.Linear(H3,H3)
        self.lin_bn4 = nn.BatchNorm1d(num_features=H3)
        

#         # Latent vectors mu and sigma
        self.fc1 = nn.Linear(H3, latent_dim)
        self.bn1 = nn.BatchNorm1d(num_features=latent_dim)
        self.fc21 = nn.Linear(latent_dim, latent_dim)
        self.fc22 = nn.Linear(latent_dim, latent_dim)

#         # Sampling vector
        self.fc3 = nn.Linear(latent_dim, latent_dim)
        self.fc_bn3 = nn.BatchNorm1d(latent_dim)
        self.fc4 = nn.Linear(latent_dim, H3)
        self.fc_bn4 = nn.BatchNorm1d(H3)

#         # Decoder
        self.linear5=nn.Linear(H3,H3)
        self.lin_bn5 = nn.BatchNorm1d(num_features=H3)
        self.linear6=nn.Linear(H3,H2)
        self.lin_bn6 = nn.BatchNorm1d(num_features=H2)
        self.linear7=nn.Linear(H2,H)
        self.lin_bn7 = nn.BatchNorm1d(num_features=H)
        self.linear8=nn.Linear(H,D_in)
        self.lin_bn8 = nn.BatchNorm1d(num_features=D_in)
        self.prev_seed =random.uniform(0,1)

        self.relu = nn.ReLU()
        self.Tanh = nn.Tanh()
        
        self.fn=fn

    def encode(self, x):
        
        if self.fn=='tanh':
            lin1 = self.Tanh(self.lin_bn1(self.linear1(x)))
            lin2 =self.Tanh(self.lin_bn2(self.linear2(lin1)))
            lin3 = self.Tanh(self.lin_bn3(self.linear3(lin2)))
            lin4 = self.Tanh(self.lin_bn4(self.linear4(lin3)))
        if self.fn=='relu':
            lin1 = self.relu(self.lin_bn1(self.linear1(x)))
            lin2 =self.relu(self.lin_bn2(self.linear2(lin1)))
            lin3 = self.relu(self.lin_bn3(self.linear3(lin2)))
            lin4 = self.relu(self.lin_bn4(self.linear4(lin3)))

        fc1 = F.relu(self.bn1(self.fc1(lin4)))

        r1 = self.fc21(fc1)
        r2 = self.fc22(fc1)

        return r1, r2

    def decode(self, z):
        
        if self.fn=='tanh':
            fc3 = self.Tanh(self.fc_bn3(self.fc3(z)))
            fc4 =self.Tanh(self.fc_bn4(self.fc4(fc3)))

            lin5 =self.Tanh(self.lin_bn5(self.linear5(fc4)))
            lin6 = self.Tanh(self.lin_bn6(self.linear6(lin5)))
            lin7 = self.Tanh(self.lin_bn7(self.linear7(lin6)))
        if self.fn=='relu':
            fc3 = self.relu(self.fc_bn3(self.fc3(z)))
            fc4 =self.relu(self.fc_bn4(self.fc4(fc3)))

            lin5 =self.relu(self.lin_bn5(self.linear5(fc4)))
            lin6 = self.relu(self.lin_bn6(self.linear6(lin5)))
            lin7 = self.relu(self.lin_bn7(self.linear7(lin6)))
        
        return self.lin_bn8(self.linear8(lin7))
    
    

    def forward(self, x):
        mu,sigma = self.encode(x)
        prev_seed = self.prev_seed
#         print('vae prev_seed',prev_seed)
        z,self.prev_seed = reparameterize(mu,sigma,prev_seed)
#         print("z",z.shape)
#         znew = sampling_vector(z,H)
        return self.decode(z),mu,sigma

In [44]:

def get_chaotic_distribution(latent_dims,previous_seed):
      # Logistic Map
#         print("latent_dims_cha",latent_dims[1])
        noise=[]
        x0=previous_seed
#         print("prev_seed",x0)
      #print("chaotic on cards ")
      #print(latent_dims,type(latent_dims))
      #print(previous_seed)

        for i in range(latent_dims[1]):
            x1=(4.0*x0*(1-x0))
            x0=x1
#             print('x1',x1)
        #print(x0,x1)
            noise.append(x1)


        t_noise=torch.FloatTensor(noise)
#         print("t_noise",t_noise)
        return t_noise,x0

def reparameterize(mu,sigma,prev_seed):
        random_seed = 1
        torch.manual_seed(random_seed)

        std = sigma.mul(0.5).exp_()
        eps = Variable(std.data.new(std.size())).normal_()
        eps_row = eps.shape[0]
        eps_col = eps.shape[1]
#         print('eps_row',eps_row,'eps_col',eps_col)
        eps = eps.reshape(1,-1)
#         print("eps shape",eps.shape)
        chaot,prev_seed = get_chaotic_distribution(eps.shape,prev_seed)
#         print("chaot_shape",chaot.shape)
        chaot = chaot.reshape(eps_row,eps_col)
#         print("chaot_reshaped",chaot.shape)
#         print("reparameter",eps.mul(std).add_(mu))
        return chaot.mul(std).add_(mu),prev_seed


In [45]:
class sampling_vector(nn.Module):
    def __init__(self, z,H2):
        super().__init__()
        self.z = z
        self.fc3 = nn.Linear(self.z,H2)
        self.fc_bn3 = nn.BatchNorm1d(H2)
#         self.fc4 = nn.Linear(latent_dims, H2)
#         self.fc_bn4 = nn.BatchNorm1d(H2)
        self.relu = nn.ReLU()
    def forward(self,z):
        fc3 = self.relu(self.fc_bn3(self.fc3(z)))
        return fc3

In [46]:
class customLoss(nn.Module):
    def __init__(self):
        super(customLoss, self).__init__()
        self.mse_loss = nn.MSELoss(reduction="sum")

    def forward(self, x_recon, x, mu, logvar):
        loss_MSE = self.mse_loss(x_recon, x)
        loss_KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

        return loss_MSE + loss_KLD

In [47]:
epochs = 1500
log_interval = 50
val_losses = []
train_losses = []
test_losses = []


def train(epoch):
    model.train()
    train_loss = 0
    
    for batch_idx, data in enumerate(trainloader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_mse(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if epoch % 100 == 0:
            print('====> Epoch: {} Average training loss: {:.4f}'.format(
            epoch, train_loss / len(trainloader.dataset)))
        train_losses.append(train_loss / len(trainloader.dataset))
        
def test(epoch):
    with torch.no_grad():
        test_loss = 0
        for batch_idx, data in enumerate(testloader):
            data = data.to(device)
            optimizer.zero_grad()
            recon_batch = model(data)
            loss = loss_mse(recon_batch, data, mu, logvar)
            test_loss += loss.item()
            if epoch % 100 == 0:
                print('====> Epoch: {} Average test loss: {:.4f}'.format(
                    epoch, test_loss / len(testloader.dataset)))
            test_losses.append(test_loss / len(testloader.dataset))

In [48]:
loss_mse = customLoss()

In [49]:
D_in = trainloader.dataset.x.shape[1]

model = VAE2(D_in,2,'relu').to(device)
optimizer = torch.optim.Adagrad(model.parameters(), lr = 0.01)

In [50]:
start = time.time()
epochs=300
for epoch in range(1, epochs + 1):
    train(epoch)
#     test(epoch)
elapsed_time = time.time()-start
print("total_time taken is :",elapsed_time)
#     test(epoch)

====> Epoch: 100 Average training loss: 6.1700
====> Epoch: 200 Average training loss: 4.8804
====> Epoch: 300 Average training loss: 4.0591
total_time taken is : 54.6328341960907


In [51]:
print(train_losses[len(train_losses)-1])

4.059070870535714


## HYPER-PARAMETER TUNING

In [52]:
global model
global optimizer

model = VAE2(D_in,2,'relu').to(device)
optimizer = torch.optim.Adagrad(model.parameters(), lr = 0.01)

In [53]:
epochs=[1800]
hl=[2,3,4]
lr=[0.0001,0.01,0.001]
mm=[0.001,0.01]
act=['relu','tanh']
opt=['adam','adagrad','sgd']
ldm=[2,3,4]

min_loss=10000
MODEL=None
BESTCOMB=None

for e in epochs:
    for h in hl:
        for l in lr:
            for m in mm:
                for a in act:
                    for o in opt:
                        for ld in ldm:
                            
                            D_in = trainloader.dataset.x.shape[1]
                            
                            if hl==2:
                                model = VAE2(D_in,ld,a).to(device)
                            if hl==3:
                                model = VAE3(D_in,ld,a).to(device)
                            if hl==4:
                                model = VAE4(D_in,ld,a).to(device)
                            
                            if opt=='adam':
                                optimizer = optim.Adam(model.parameters(), lr=l)
                            if opt=='sgd':
                                optimizer = optim.SGD(model.parameters(), lr=l,momentum=m)
                            if opt=='adagrad':
                                optimizer = torch.optim.Adagrad(model.parameters(), lr = l)
                            
                            start = time.time()
                            for epoch in range(1, e+1):
                                train(epoch)
                            #     test(epoch)
                            elapsed_time = time.time()-start
                            #print("total_time taken is :",elapsed_time)

                            cur_loss=train_losses[len(train_losses)-1]
                            if cur_loss < min_loss:
                                min_loss=cur_loss
                                MODEL=model
                                BESTCOMB=[e,h,l,m,a,o,ld]
                            
                            
                            ## junk --ignore
                            model = VAE2(D_in,2,'relu').to(device)
                            optimizer = torch.optim.Adagrad(model.parameters(), lr = 0.01)

====> Epoch: 100 Average training loss: 5.9166
====> Epoch: 200 Average training loss: 4.7501
====> Epoch: 300 Average training loss: 3.9874
====> Epoch: 400 Average training loss: 3.4448
====> Epoch: 500 Average training loss: 3.0195
====> Epoch: 600 Average training loss: 2.6879
====> Epoch: 700 Average training loss: 2.4348
====> Epoch: 800 Average training loss: 2.1997
====> Epoch: 900 Average training loss: 2.0237
====> Epoch: 1000 Average training loss: 1.8572
====> Epoch: 1100 Average training loss: 1.7204
====> Epoch: 1200 Average training loss: 1.6014
====> Epoch: 1300 Average training loss: 1.5029
====> Epoch: 1400 Average training loss: 1.4031
====> Epoch: 1500 Average training loss: 1.3280
====> Epoch: 1600 Average training loss: 1.2667
====> Epoch: 1700 Average training loss: 1.1965
====> Epoch: 1800 Average training loss: 1.1325
====> Epoch: 100 Average training loss: 5.9155
====> Epoch: 200 Average training loss: 4.7100
====> Epoch: 300 Average training loss: 3.9408
====

====> Epoch: 1200 Average training loss: 1.6481
====> Epoch: 1300 Average training loss: 1.5296
====> Epoch: 1400 Average training loss: 1.4436
====> Epoch: 1500 Average training loss: 1.3563
====> Epoch: 1600 Average training loss: 1.2694
====> Epoch: 1700 Average training loss: 1.1954
====> Epoch: 1800 Average training loss: 1.1294
====> Epoch: 100 Average training loss: 5.9266
====> Epoch: 200 Average training loss: 4.7527
====> Epoch: 300 Average training loss: 4.0012
====> Epoch: 400 Average training loss: 3.4519
====> Epoch: 500 Average training loss: 3.0326
====> Epoch: 600 Average training loss: 2.6929
====> Epoch: 700 Average training loss: 2.4339
====> Epoch: 800 Average training loss: 2.2021
====> Epoch: 900 Average training loss: 2.0291
====> Epoch: 1000 Average training loss: 1.8663
====> Epoch: 1100 Average training loss: 1.7162
====> Epoch: 1200 Average training loss: 1.6012
====> Epoch: 1300 Average training loss: 1.4897
====> Epoch: 1400 Average training loss: 1.4143
=

====> Epoch: 500 Average training loss: 3.0219
====> Epoch: 600 Average training loss: 2.6949
====> Epoch: 700 Average training loss: 2.4322
====> Epoch: 800 Average training loss: 2.1958
====> Epoch: 900 Average training loss: 1.9960
====> Epoch: 1000 Average training loss: 1.8475
====> Epoch: 1100 Average training loss: 1.6872
====> Epoch: 1200 Average training loss: 1.5855
====> Epoch: 1300 Average training loss: 1.4511
====> Epoch: 1400 Average training loss: 1.3860
====> Epoch: 1500 Average training loss: 1.3101
====> Epoch: 1600 Average training loss: 1.2797
====> Epoch: 1700 Average training loss: 1.1790
====> Epoch: 1800 Average training loss: 1.1378
====> Epoch: 100 Average training loss: 5.9274
====> Epoch: 200 Average training loss: 4.7452
====> Epoch: 300 Average training loss: 4.0150
====> Epoch: 400 Average training loss: 3.4592
====> Epoch: 500 Average training loss: 3.0639
====> Epoch: 600 Average training loss: 2.7290
====> Epoch: 700 Average training loss: 2.4712
====

====> Epoch: 1600 Average training loss: 1.2478
====> Epoch: 1700 Average training loss: 1.1846
====> Epoch: 1800 Average training loss: 1.1269
====> Epoch: 100 Average training loss: 5.9197
====> Epoch: 200 Average training loss: 4.7473
====> Epoch: 300 Average training loss: 3.9889
====> Epoch: 400 Average training loss: 3.4243
====> Epoch: 500 Average training loss: 3.0121
====> Epoch: 600 Average training loss: 2.6861
====> Epoch: 700 Average training loss: 2.4304
====> Epoch: 800 Average training loss: 2.2011
====> Epoch: 900 Average training loss: 2.0126
====> Epoch: 1000 Average training loss: 1.8666
====> Epoch: 1100 Average training loss: 1.7060
====> Epoch: 1200 Average training loss: 1.5955
====> Epoch: 1300 Average training loss: 1.4976
====> Epoch: 1400 Average training loss: 1.3981
====> Epoch: 1500 Average training loss: 1.3310
====> Epoch: 1600 Average training loss: 1.2542
====> Epoch: 1700 Average training loss: 1.1873
====> Epoch: 1800 Average training loss: 1.1424
=

====> Epoch: 900 Average training loss: 1.9946
====> Epoch: 1000 Average training loss: 1.8344
====> Epoch: 1100 Average training loss: 1.7095
====> Epoch: 1200 Average training loss: 1.5832
====> Epoch: 1300 Average training loss: 1.4841
====> Epoch: 1400 Average training loss: 1.3844
====> Epoch: 1500 Average training loss: 1.3157
====> Epoch: 1600 Average training loss: 1.2385
====> Epoch: 1700 Average training loss: 1.1729
====> Epoch: 1800 Average training loss: 1.1226
====> Epoch: 100 Average training loss: 5.9251
====> Epoch: 200 Average training loss: 4.7444
====> Epoch: 300 Average training loss: 4.0104
====> Epoch: 400 Average training loss: 3.4604
====> Epoch: 500 Average training loss: 3.0487
====> Epoch: 600 Average training loss: 2.7098
====> Epoch: 700 Average training loss: 2.4503
====> Epoch: 800 Average training loss: 2.2211
====> Epoch: 900 Average training loss: 2.0337
====> Epoch: 1000 Average training loss: 1.8690
====> Epoch: 1100 Average training loss: 1.7346
==

...

====> Epoch: 1400 Average training loss: 1.3970
====> Epoch: 1500 Average training loss: 1.3144
====> Epoch: 1600 Average training loss: 1.2547
====> Epoch: 1700 Average training loss: 1.1934
====> Epoch: 1800 Average training loss: 1.1431
====> Epoch: 100 Average training loss: 5.9194
====> Epoch: 200 Average training loss: 4.7515
====> Epoch: 300 Average training loss: 4.0011
====> Epoch: 400 Average training loss: 3.4544
====> Epoch: 500 Average training loss: 3.0465
====> Epoch: 600 Average training loss: 2.7165
====> Epoch: 700 Average training loss: 2.4398
====> Epoch: 800 Average training loss: 2.2151
====> Epoch: 900 Average training loss: 2.0456
====> Epoch: 1000 Average training loss: 1.8772
====> Epoch: 1100 Average training loss: 1.7316
====> Epoch: 1200 Average training loss: 1.6192
====> Epoch: 1300 Average training loss: 1.1539
====> Epoch: 1400 Average training loss: 0.9633
====> Epoch: 1500 Average training loss: 0.8763
====> Epoch: 1600 Average training loss: 0.8144
=

====> Epoch: 700 Average training loss: 2.4442
====> Epoch: 800 Average training loss: 2.2271
====> Epoch: 900 Average training loss: 2.0334
====> Epoch: 1000 Average training loss: 1.8744
====> Epoch: 1100 Average training loss: 1.7338
====> Epoch: 1200 Average training loss: 1.6129
====> Epoch: 1300 Average training loss: 1.5075
====> Epoch: 1400 Average training loss: 1.4156
====> Epoch: 1500 Average training loss: 1.3362
====> Epoch: 1600 Average training loss: 1.2655
====> Epoch: 1700 Average training loss: 1.2035
====> Epoch: 1800 Average training loss: 1.1422
====> Epoch: 100 Average training loss: 5.9101
====> Epoch: 200 Average training loss: 4.7281
====> Epoch: 300 Average training loss: 3.9759
====> Epoch: 400 Average training loss: 3.4514
====> Epoch: 500 Average training loss: 3.0420
====> Epoch: 600 Average training loss: 2.7263
====> Epoch: 700 Average training loss: 2.2541
====> Epoch: 800 Average training loss: 2.0189
====> Epoch: 900 Average training loss: 1.8236
====

====> Epoch: 1800 Average training loss: 1.1587
====> Epoch: 100 Average training loss: 5.9426
====> Epoch: 200 Average training loss: 4.7236
====> Epoch: 300 Average training loss: 3.9735
====> Epoch: 400 Average training loss: 3.4271
====> Epoch: 500 Average training loss: 3.0364
====> Epoch: 600 Average training loss: 2.6862
====> Epoch: 700 Average training loss: 2.4163
====> Epoch: 800 Average training loss: 2.2016
====> Epoch: 900 Average training loss: 2.0116
====> Epoch: 1000 Average training loss: 1.8521
====> Epoch: 1100 Average training loss: 1.7092
====> Epoch: 1200 Average training loss: 1.5914
====> Epoch: 1300 Average training loss: 1.4884
====> Epoch: 1400 Average training loss: 1.3883
====> Epoch: 1500 Average training loss: 1.3208
====> Epoch: 1600 Average training loss: 1.2522
====> Epoch: 1700 Average training loss: 1.1985
====> Epoch: 1800 Average training loss: 1.1094
====> Epoch: 100 Average training loss: 5.9074
====> Epoch: 200 Average training loss: 4.7247
===

====> Epoch: 1100 Average training loss: 1.7303
====> Epoch: 1200 Average training loss: 1.6188
====> Epoch: 1300 Average training loss: 1.5125
====> Epoch: 1400 Average training loss: 1.4274
====> Epoch: 1500 Average training loss: 1.3388
====> Epoch: 1600 Average training loss: 1.2835
====> Epoch: 1700 Average training loss: 1.2152
====> Epoch: 1800 Average training loss: 1.1513
====> Epoch: 100 Average training loss: 5.9059
====> Epoch: 200 Average training loss: 4.7518
====> Epoch: 300 Average training loss: 3.9934
====> Epoch: 400 Average training loss: 3.4515
====> Epoch: 500 Average training loss: 3.0333
====> Epoch: 600 Average training loss: 2.7081
====> Epoch: 700 Average training loss: 2.4385
====> Epoch: 800 Average training loss: 2.2067
====> Epoch: 900 Average training loss: 2.0131
====> Epoch: 1000 Average training loss: 1.8654
====> Epoch: 1100 Average training loss: 1.7201
====> Epoch: 1200 Average training loss: 1.6016
====> Epoch: 1300 Average training loss: 1.4933
=

====> Epoch: 400 Average training loss: 3.4497
====> Epoch: 500 Average training loss: 3.0385
====> Epoch: 600 Average training loss: 2.7065
====> Epoch: 700 Average training loss: 2.4274
====> Epoch: 800 Average training loss: 2.2150
====> Epoch: 900 Average training loss: 2.0163
====> Epoch: 1000 Average training loss: 1.8489
====> Epoch: 1100 Average training loss: 1.7060
====> Epoch: 1200 Average training loss: 1.5948
====> Epoch: 1300 Average training loss: 1.4933
====> Epoch: 1400 Average training loss: 1.4041
====> Epoch: 1500 Average training loss: 1.3354
====> Epoch: 1600 Average training loss: 1.2546
====> Epoch: 1700 Average training loss: 1.1907
====> Epoch: 1800 Average training loss: 1.1326
====> Epoch: 100 Average training loss: 5.9275
====> Epoch: 200 Average training loss: 4.7026
====> Epoch: 300 Average training loss: 3.9472
====> Epoch: 400 Average training loss: 3.4117
====> Epoch: 500 Average training loss: 3.0125
====> Epoch: 600 Average training loss: 2.6795
====

In [54]:
BESTCOMB

[1800, 4, 0.001, 0.001, 'relu', 'adam', 3]

In [55]:
MODEL

VAE2(
  (linear1): Linear(in_features=8, out_features=10, bias=True)
  (lin_bn1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear2): Linear(in_features=10, out_features=15, bias=True)
  (lin_bn2): BatchNorm1d(15, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=15, out_features=2, bias=True)
  (bn1): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc21): Linear(in_features=2, out_features=2, bias=True)
  (fc22): Linear(in_features=2, out_features=2, bias=True)
  (fc3): Linear(in_features=2, out_features=2, bias=True)
  (fc_bn3): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc4): Linear(in_features=2, out_features=15, bias=True)
  (fc_bn4): BatchNorm1d(15, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear4): Linear(in_features=15, out_features=10, bias=True)
  (lin_bn4): BatchNorm1d(10, eps=1e-05, momentum=0.1

In [56]:
x_hat = []
with torch.no_grad():
    for batch_idx, data in enumerate(trainloader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = MODEL(data)
        x_hat.append(recon_batch)


In [57]:
x_hat

[tensor([[ 0.4852,  0.3759,  0.4209,  ...,  0.9563,  1.0404,  0.3530],
         [ 0.9622,  0.1287,  0.7415,  ...,  0.8578, -0.0386,  0.5419],
         [ 0.4835,  0.3758,  0.4205,  ...,  0.9413,  1.0325,  0.3541],
         ...,
         [ 0.4870,  0.3760,  0.4214,  ...,  0.9723,  1.0488,  0.3518],
         [ 0.4850,  0.3759,  0.4209,  ...,  0.9548,  1.0396,  0.3531],
         [ 0.3611,  0.3677,  0.3891,  ..., -0.1235,  0.4736,  0.4316]])]

In [58]:
d1 = pd.DataFrame(np.array(x_hat[0]))

In [59]:
d1

,0,1,2,3,4,5,6,7
0,0.485203,0.375921,0.420938,0.344452,0.068266,0.956314,1.040355,0.352975
1,0.962158,0.128706,0.741460,0.161833,0.355660,0.857796,-0.038624,0.541858
2,0.483472,0.375809,0.420497,0.344664,0.069110,0.941343,1.032497,0.354062
3,0.484332,0.375867,0.420718,0.344558,0.068690,0.948755,1.036387,0.353522
4,0.486555,0.376010,0.421284,0.344286,0.067601,0.968011,1.046494,0.352118
...,...,...,...,...,...,...,...,...
6995,0.483483,0.375809,0.420499,0.344661,0.069110,0.941358,1.032504,0.354062
6996,0.488356,0.376137,0.421747,0.344074,0.066705,0.983783,1.054774,0.350969
6997,0.487042,0.376049,0.421413,0.344231,0.067354,0.972329,1.048762,0.351806
6998,0.485027,0.375913,0.420893,0.344475,0.068351,0.954782,1.039551,0.353082


In [60]:
# d2

In [61]:
org =pd.DataFrame(np.array(trainloader.dataset.x))

In [62]:
# pd.DataFrame(np.array(testloader.dataset.x)).mean()


In [63]:
org.std()

0    0.192962
1    0.141110
2    0.287587
3    0.248447
4    0.192987
5    0.456147
6    0.499946
7    0.286641
dtype: float32

In [64]:
syn_test = pd.concat([d1],ignore_index = True)

In [65]:
syn_test.shape

(7000, 8)

In [66]:
syn_test.std()

0    0.246317
1    0.123391
2    0.149038
3    0.074345
4    0.114435
5    0.468882
6    0.463726
7    0.188770
dtype: float32

In [67]:
# d1.to_csv('vae-wnn-yes.csv')

## Evasion Attack

In [72]:
x_hat = []
with torch.no_grad():
    for batch_idx, data in enumerate(testloader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = MODEL(data)
        x_hat.append(recon_batch)

d1 = pd.DataFrame(np.array(x_hat[0]))
eva_test = pd.concat([d1],ignore_index = True)

In [74]:
DATA_PATH = "../../../datasets/Churn_Modelling.csv"

## Dataset Overview

df_base = pd.read_csv(DATA_PATH, sep=',')
df_base.head()

cols = df_base.columns

df = pd.read_csv(DATA_PATH, sep=',')
# replace nan with -99
y = df['Exited']
df.drop(['RowNumber', 'CustomerId', 'Surname','Geography',
   'Gender','Exited'],axis=1,inplace=True)
#df = df.fillna(-99)
#df = df.fillna(-99)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data=scaler.fit_transform(df)
df1=pd.DataFrame(data)
# randomly split
X_train, X_test,Y_train,Y_test = train_test_split(df1, y,test_size=0.3, stratify=y, random_state=4)


Y_train.reset_index(drop = True);Y_test.reset_index(drop = True)

## Models

## Logistic Regression

print("LOGISTIC REGRESSION")

np.random.seed(42)

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

lr.fit(X_train,Y_train)

preds = lr.predict(eva_test)

#roc_auc_score(Y_test,ypred_lr)

from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(Y_test, preds).ravel()

p1 =(tp+fn)
sen = (tp/p1)

p2 = (tn + fp)
spec = tn/p2

p3 = sen + spec
auc_score = 0.5*p3

print(tn, fp, fn, tp)

print(sen)
print(spec)
print(auc_score)

print("Decision Tree")

np.random.seed(42)

from sklearn.tree import  DecisionTreeClassifier

logi = DecisionTreeClassifier()
logi.fit(X_train,Y_train)

preds =logi.predict(eva_test)

from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(Y_test, preds).ravel()

p1 =(tp+fn)
sen = (tp/p1)

p2 = (tn + fp)
spec = tn/p2

p3 = sen + spec
auc_score = 0.5*p3

print(tn, fp, fn, tp)

print(sen)
print(spec)
print(auc_score)


print("LGBM Classifier")

np.random.seed(42)

from lightgbm import LGBMClassifier

# parameters = {'n_estimators': [25,50,100,150,175,200]}
parameters = {'criterion' : ["gini", "entropy"],
              'n_estimators': [25,50,100,150,175,200]}
# best_model=None
# best_score=0.0

# for ne in parameters['n_estimators']:
#     gb = LGBMClassifier(n_estimators=ne)
#     gb.fit(X_train,Y_train)
#     y_predg=gb.predict(X_test)

#     #accuracy=accuracy_score(Y_test,y_predg)
#     auc=roc_auc_score(Y_test,y_predg)
#     # print('Combination',c,ne)
#     #print('AUC',auc)

#     if auc > best_score:
#         best_model=gb
#         best_score=auc
        
gb = LGBMClassifier(n_estimators=150)
gb.fit(X_train,Y_train)
preds=gb.predict(eva_test)

from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(Y_test, preds).ravel()

p1 =(tp+fn)
sen = (tp/p1)

p2 = (tn + fp)
spec = tn/p2

p3 = sen + spec
auc_score = 0.5*p3

print(tn, fp, fn, tp)

print(sen)
print(spec)
print(auc_score)



print("XGB Classifier")

from xgboost import XGBClassifier
xgb = XGBClassifier()

xgb_model = xgb.fit(X_train,Y_train)
preds = xgb_model.predict(eva_test)


from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(Y_test, preds).ravel()

p1 =(tp+fn)
sen = (tp/p1)

p2 = (tn + fp)
spec = tn/p2

p3 = sen + spec
auc_score = 0.5*p3

print(tn, fp, fn, tp)

print(sen)
print(spec)
print(auc_score)



LOGISTIC REGRESSION
2389 0 611 0
0.0
1.0
0.5
Decision Tree
1987 402 461 150
0.24549918166939444
0.8317287568020092
0.5386139692357018
LGBM Classifier
2113 276 509 102
0.16693944353518822
0.884470489744663
0.5257049666399256
XGB Classifier
2021 368 469 142
0.23240589198036007
0.845960652992884
0.539183272486622


## Poison Attack

In [75]:
x_hat = []
with torch.no_grad():
    for batch_idx, data in enumerate(trainloader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = MODEL(data)
        x_hat.append(recon_batch)

d1 = pd.DataFrame(np.array(x_hat[0]))
eva_train = pd.concat([d1],ignore_index = True)

In [76]:
DATA_PATH = "../../../datasets/Churn_Modelling.csv"

## Dataset Overview

df_base = pd.read_csv(DATA_PATH, sep=',')
df_base.head()

cols = df_base.columns

df = pd.read_csv(DATA_PATH, sep=',')
# replace nan with -99
y = df['Exited']
df.drop(['RowNumber', 'CustomerId', 'Surname','Geography',
   'Gender','Exited'],axis=1,inplace=True)
#df = df.fillna(-99)
#df = df.fillna(-99)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data=scaler.fit_transform(df)
df1=pd.DataFrame(data)
# randomly split
X_train, X_test,Y_train,Y_test = train_test_split(df1, y,test_size=0.3, random_state=4)


Y_train.reset_index(drop = True);Y_test.reset_index(drop = True)

## Models

## Logistic Regression

print("LOGISTIC REGRESSION")

np.random.seed(42)

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

lr.fit(eva_train,Y_train)

preds = lr.predict(X_test)

#roc_auc_score(Y_test,ypred_lr)

from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(Y_test, preds).ravel()

p1 =(tp+fn)
sen = (tp/p1)

p2 = (tn + fp)
spec = tn/p2

p3 = sen + spec
auc_score = 0.5*p3

print(tn, fp, fn, tp)

print(sen)
print(spec)
print(auc_score)

print("Decision Tree")

np.random.seed(42)

from sklearn.tree import  DecisionTreeClassifier

logi = DecisionTreeClassifier()
logi.fit(eva_train,Y_train)

preds =logi.predict(X_test)

from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(Y_test, preds).ravel()

p1 =(tp+fn)
sen = (tp/p1)

p2 = (tn + fp)
spec = tn/p2

p3 = sen + spec
auc_score = 0.5*p3

print(tn, fp, fn, tp)

print(sen)
print(spec)
print(auc_score)


print("LGBM Classifier")

np.random.seed(42)

from lightgbm import LGBMClassifier

# parameters = {'n_estimators': [25,50,100,150,175,200]}
# parameters = {'criterion' : ["gini", "entropy"],
#               'n_estimators': [25,50,100,150,175,200]}
# best_model=None
# best_score=0.0

# for ne in parameters['n_estimators']:
#     gb = LGBMClassifier(n_estimators=ne)
#     gb.fit(X_train,Y_train)
#     y_predg=gb.predict(X_test)

#     #accuracy=accuracy_score(Y_test,y_predg)
#     auc=roc_auc_score(Y_test,y_predg)
#     # print('Combination',c,ne)
#     #print('AUC',auc)

#     if auc > best_score:
#         best_model=gb
#         best_score=auc
        
gb = LGBMClassifier(n_estimators=150)
gb.fit(eva_train,Y_train)
preds=gb.predict(X_test)

from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(Y_test, preds).ravel()

p1 =(tp+fn)
sen = (tp/p1)

p2 = (tn + fp)
spec = tn/p2

p3 = sen + spec
auc_score = 0.5*p3

print(tn, fp, fn, tp)

print(sen)
print(spec)
print(auc_score)



print("XGB Classifier")

from xgboost import XGBClassifier
xgb = XGBClassifier()

xgb_model = xgb.fit(eva_train,Y_train)
preds = xgb_model.predict(X_test)


from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(Y_test, preds).ravel()

p1 =(tp+fn)
sen = (tp/p1)

p2 = (tn + fp)
spec = tn/p2

p3 = sen + spec
auc_score = 0.5*p3

print(tn, fp, fn, tp)

print(sen)
print(spec)
print(auc_score)



LOGISTIC REGRESSION
2416 0 584 0
0.0
1.0
0.5
Decision Tree
1183 1233 289 295
0.5051369863013698
0.4896523178807947
0.49739465209108225
LGBM Classifier
1962 454 511 73
0.125
0.8120860927152318
0.4685430463576159
XGB Classifier
1292 1124 329 255
0.4366438356164384
0.5347682119205298
0.4857060237684841
